# Debug （基于python语言)

## 传统的调试姿势
1. print / logging
2. pdb (打断点)
    
### 不好用的地方:
- 适用于开发过程
- 无法持续追踪问题
- 复杂的代码结构进行调试及其困难




## sys.settrace
原理: 开发者自定义一个'调试器', 通过settrace进行注册, settrace能够设置系统的跟踪功能。使得在程序运行时获取相关信息，实现动态debug.


### Quick Strat

In [ ]:
import sys

def foo():
    print("hello")
    print("world")


def tracer(frame, event, args):
    print({'event': event, 'args': args, 'context': frame.f_code.co_name})
    return tracer
    
if __name__ == '__main__':
    sys.settrace(tracer)
    foo()


    
    {'event': 'call', 'args': None, 'context': 'foo'}
    {'event': 'line', 'args': None, 'context': 'foo'}
    hello
    {'event': 'line', 'args': None, 'context': 'foo'}
    world
    {'event': 'return', 'args': None, 'context': 'foo'}

#### 事件枚举  
事件 |处理器| 栈帧位置|参数|返回值
 :-| :-| :-|:-|:- 
 call  | global tracer | 函数定义行 | None | local tracer / None
 line  | local tracer | 执行当前行 | None | local tracer / None
 return| local tracer | 返回语句所在行 | 返回值 / None | Ignored
 exception| local tracer| 异常冒泡所在行 | (exc, val, tb) | local tracer / None   
 



#### frame(栈帧)
栈帧就是存储在用户栈上的（当然内核栈同样适用）每一次函数调用涉及的相关信息的记录单元， 栈帧的主要作用是**用来控制和保存一个过程的所有信息的**。

有关python的栈帧数据结构的详细介绍， 可以查看[Python3 inspect module](https://docs.python.org/3/library/inspect.html)

**几个常用的信息:**

属性|保存的信息
:-:| -:
frame.f_lineno | current line number in Python source code
frmae.f_locals | local namespace seen by this frame
frmae.f_code.co_name | name with which this code object was defined
frame.f_code.co_filename | name of file in which this code object was created

### 应用 
#### 变量追踪

In [ ]:
import sys

def quick_sort(vector):
    if len(vector) < 2:
        return vector
    
    else:
        flag = vector[0]
        less = [i for i in vector[1:] if i <= flag]
        big = [i for i in vector[1:] if i > flag]
        final = quick_sort(less) + [flag] + quick_sort(big)
        return final
    
def tracer(frame, event, args):
    if 'final' in frame.f_locals.keys():
        print(f'{frame.f_locals}')
    return tracer

if __name__ == '__main__':
    sys.settrace(tracer)
    test = [1, 9, 2, 4, 1, 2]
    quick_sort(test)
    


    
    {'vector': [1, 9, 2, 4, 1, 2]}
    {'vector': [1, 9, 2, 4, 1, 2], 'flag': 1}
    {'vector': [1, 9, 2, 4, 1, 2], 'flag': 1, 'less': [1]}
    {'vector': [1, 9, 2, 4, 1, 2], 'flag': 1, 'less': [1], 'big': [9, 2, 4, 2]}
    ...
    ...
    {'vector': [9, 2, 4, 2], 'flag': 9, 'less': [2, 4, 2], 'big': [], 'final': [2, 2, 4, 9]}
    {'vector': [9, 2, 4, 2], 'flag': 9, 'less': [2, 4, 2], 'big': [], 'final': [2, 2, 4, 9]}
    {'vector': [1, 9, 2, 4, 1, 2], 'flag': 1, 'less': [1], 'big': [9, 2, 4, 2], 'final': [1, 1, 2, 2, 4, 9]}
    {'vector': [1, 9, 2, 4, 1, 2], 'flag': 1, 'less': [1], 'big': [9, 2, 4, 2], 'final': [1, 1, 2, 2, 4, 9]}

##### 当年 1.6 亿美金估值的公司—— Digg 是如何被一句 Python 函数可变默参 毁掉的  [转自v2ex](https://www.v2ex.com/t/467817)

      2011 年，Google 推出「 Panda 」 机制动摇了很多老的 SEO 手段，digg 流量被腰斩。推出 DiggV4 作战计划。经过紧张的开发发布，不过访客页面没问题，已登录用户打不开 MyNews 页面。开发不得不用临时手段把登录用户的默认页面改成 TopNews         
      MyNews 只能通过不断重启进程才能短暂修复。初期以为是 cassandra 的缓存击穿了 memcache，后来加紧用 redis 重写了，还是得几个小时重启一次。   
      折腾了一个月之后.....
      终于发现原因了：API 服务器是 tornado 写的名字叫 Bobtail。里面最常用的函数是：
<p> <code>    def get_user_by_ids(ids=[]):
                ....            </code></p>
      
**然后这个ids通过一直append，直到撑爆内存。**
      所以这个 MyNews 功能也渐渐用的人少，因为没法定制化看新闻，后来，大家都不去 diggv4 而去 reddit 了。。最后，digg 以 50w 美金被收了。



#### 函数过程追踪

In [2]:
import asyncio
import sys


async def func1(future, N):
    await asyncio.sleep(N)
    future.set_result('This is func1')


async def func2(future, N):
    await asyncio.sleep(N)
    future.set_result('This is func2')


def got_result(future):
    print(future.result())


def tracer(frame, event, args):
    if event == 'call' and frame.f_code.co_filename == 'test.py':
        print({'filename': frame.f_code.co_filename,
               'lineno': frame.f_lineno,
               'context': frame.f_code.co_name})
        return tracer
    return tracer


if __name__ == '__main__':
    sys.settrace(tracer)

    loop = asyncio.get_event_loop()

    future1 = asyncio.Future()
    future2 = asyncio.Future()

    task = [asyncio.ensure_future(func1(future1, 3)),
            asyncio.ensure_future(func2(future2, 2))]

    future1.add_done_callback(got_result)
    future2.add_done_callback(got_result)

    loop.run_until_complete(asyncio.wait(task))
    loop.close()


    
    {'filename': 'test.py', 'lineno': 5, 'context': 'func1'}
    {'filename': 'test.py', 'lineno': 10, 'context': 'func2'}
    {'filename': 'test.py', 'lineno': 11, 'context': 'func2'}
    {'filename': 'test.py', 'lineno': 15, 'context': 'got_result'}
    This is func2
    {'filename': 'test.py', 'lineno': 6, 'context': 'func1'}
    {'filename': 'test.py', 'lineno': 15, 'context': 'got_result'}
    This is func1
    ...